---
# __Logistic Regression with Trailing 5-Match xG Averages and Time-Based Splits__
---

# Data Loading
## 1. Load the Dataset and Parse Match Dates

- First, we load the dataset and convert the 'Date' column to datetime format. This ensures we can easily filter matches by date for chronological splitting. We use pandas' to_datetime for the conversion.

In [1]:
import pandas as pd

# Load the Premier League dataset
df = pd.read_csv('PL_integrated_dataset_10years.csv')

# Convert the 'Date' column to datetime
df['Date'] = pd.to_datetime(df['Date'])

# Verify conversion (the 'Date' column should now have a datetime dtype)
print(df['Date'].dtype)


datetime64[ns]


- __Explanation__: After this step, the Date column is of type datetime (datetime64[ns]), which will allow us to perform time-based filtering and extract components like year or month easily.

---
## 2. Extract Season/Year Information from Dates
---

Next, we extract useful time information from the date. Specifically, we'll derive the year and (if needed) the season of each match. The Premier League season typically runs from August to May, crossing calendar years. We'll determine the "season year" for each match by using the match date:

If a match is in July or later (i.e., the second half of the calendar year), we'll treat it as part of the season that starts that year.

If a match is in January through June, it belongs to the season that started the previous year.

Let's create a new column to identify the season by its starting year, and also extract the calendar year and month for reference:

In [2]:
# Extract year and month from the date
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month

# Determine the season's start year for each match
df['SeasonStartYear'] = df['Date'].apply(lambda x: x.year if x.month >= 7 else x.year - 1)

# (Optional) Combine to a season label, e.g., "2023-2024"
df['Season_Label'] = df['SeasonStartYear'].astype(str) + '-' + (df['SeasonStartYear'] + 1).astype(str)
print(df[['Date', 'Season_Label']].head(3))


        Date Season_Label
0 2015-08-08    2015-2016
1 2015-08-08    2015-2016
2 2015-08-08    2015-2016


__Explanation__: We added:

- __"Year"__ and __"Month"__ columns for the calendar year and month of each match.

- __"SeasonStartYear"__ which uses the rule above (e.g., a match on 2023-08-15 gets SeasonStartYear=2023, while a match on 2024-05-10 gets SeasonStartYear=2023 because it's before July 2024).

- __"Season_Label"__ combines the start year and the next year (for instance, a match in August 2023 gets label "2023-2024").

-> This confirms which season each match belongs to, which will help ensure our splits align with season boundaries.

---
## 3. Define Chronological Train, Validation, and Test Splits
---

Now we split the dataset into three sets based on date following the __8/1/1__ principle where 8 seasons are for training the model, 1 season for  validation and 1 for testing the final performance.

- __Training set__: All matches before July 1, 2023 (i.e., up to June 30, 2023).

- __Validation set__: Matches from July 1, 2023 through June 30, 2024 (the 2023–2024 season).

- __Test set__: Matches from July 1, 2024 onward (the 2024–2025 season).

We create boolean masks for these date ranges and then apply them to the DataFrame:

In [3]:
# Define cutoff dates for splitting
train_cutoff = pd.Timestamp('2023-07-01')   # start of validation period
test_cutoff  = pd.Timestamp('2024-07-01')   # start of test period (2024-2025 season)

# Create boolean masks for each subset
train_mask = df['Date'] < train_cutoff
val_mask   = (df['Date'] >= train_cutoff) & (df['Date'] < test_cutoff)
test_mask  = df['Date'] >= test_cutoff

# Apply masks to create the splits
train_df = df[train_mask].copy()
val_df   = df[val_mask].copy()
test_df  = df[test_mask].copy()

# Check the number of matches in each subset
print("Training set matches:", len(train_df))
print("Validation set matches:", len(val_df))
print("Test set matches:", len(test_df))

Training set matches: 3014
Validation set matches: 380
Test set matches: 380


__Explanation__: We use July 1, 2023 as the boundary between training and validation, and July 1, 2024 as the boundary between validation and test. The .copy() is used to create independent DataFrames for each subset. The printed counts show how many matches fall in each set (the training set should comprise matches from 2015 up to mid-2023, and validation/test sets each roughly cover one season of 380 matches). At this stage, these splits include all matches in the date ranges, including some early-season matches that may not have trailing 5-match averages computed yet. We will handle those next.

---
## 4. Compute Trailing 5-Match Average xG Features and Filter Invalid Rows
---

We now create the features for the trailing 5-match average expected goals (xG) for both home and away teams. This feature represents each team's average xG over its last 5 matches prior to the current match.

Steps to compute trailing averages:

__1)__ Combine the home and away xG data into a single timeline for each team.

__2)__ Sort matches for each team by date.

__3)__ Compute a rolling mean of xG over the last 5 games for each team, shifting by one to ensure it's strictly the previous 5 matches (the current match's xG is not included in its own average).

__4)__ Assign these averages back to the main dataset in two new columns: home_team_avg_xG_last5 and away_team_avg_xG_last5.

__5)__ Some matches (typically at the start of a season or when a team is newly promoted) will not have 5 prior games to compute an average, resulting in NaN for those rows. We will drop those rows, as instructed, so that both home and away trailing xG values are valid.

Let's implement this:

In [4]:
import numpy as np

# Initialize new columns in the original DataFrame for trailing xG averages
df['home_team_avg_xG_last5'] = np.nan
df['away_team_avg_xG_last5'] = np.nan

# Prepare a combined dataset of team performances to compute rolling averages
home_stats = pd.DataFrame({
    'Team': df['HomeTeam'],
    'Date': df['Date'],
    'xG': df['Home_xG'],          # home team's xG in that match
    'MatchID': df.index,
    'Role': 'home'
})
away_stats = pd.DataFrame({
    'Team': df['AwayTeam'],
    'Date': df['Date'],
    'xG': df['Away_xG'],          # away team's xG in that match
    'MatchID': df.index,
    'Role': 'away'
})
combined_stats = pd.concat([home_stats, away_stats], ignore_index=True)

# Sort by team and date to ensure chronological order within each team
combined_stats.sort_values(['Team', 'Date'], inplace=True)

# Group by team and compute rolling mean of the last 5 xG values (excluding current match by shifting)
combined_stats['xG_avg_last5'] = combined_stats.groupby('Team')['xG']\
    .transform(lambda x: x.rolling(window=5, min_periods=5).mean().shift(1))

# Assign the computed averages back to the original DataFrame for each match
# We use the stored MatchID and Role to place the value in the correct home/away column
home_mask = combined_stats['Role'] == 'home'
away_mask = combined_stats['Role'] == 'away'

# For home team entries, assign to home_team_avg_xG_last5
df.loc[combined_stats.loc[home_mask, 'MatchID'], 'home_team_avg_xG_last5'] = combined_stats.loc[home_mask, 'xG_avg_last5'].values
# For away team entries, assign to away_team_avg_xG_last5
df.loc[combined_stats.loc[away_mask, 'MatchID'], 'away_team_avg_xG_last5'] = combined_stats.loc[away_mask, 'xG_avg_last5'].values

# Now update the splits DataFrames with these new features
train_df['home_team_avg_xG_last5'] = df.loc[train_df.index, 'home_team_avg_xG_last5'].values
train_df['away_team_avg_xG_last5'] = df.loc[train_df.index, 'away_team_avg_xG_last5'].values

val_df['home_team_avg_xG_last5'] = df.loc[val_df.index, 'home_team_avg_xG_last5'].values
val_df['away_team_avg_xG_last5'] = df.loc[val_df.index, 'away_team_avg_xG_last5'].values

test_df['home_team_avg_xG_last5'] = df.loc[test_df.index, 'home_team_avg_xG_last5'].values
test_df['away_team_avg_xG_last5'] = df.loc[test_df.index, 'away_team_avg_xG_last5'].values

# Drop any matches in each set where trailing xG data is not available for either team
train_df = train_df.dropna(subset=['home_team_avg_xG_last5', 'away_team_avg_xG_last5'])
val_df   = val_df.dropna(subset=['home_team_avg_xG_last5', 'away_team_avg_xG_last5'])
test_df  = test_df.dropna(subset=['home_team_avg_xG_last5', 'away_team_avg_xG_last5'])

# Confirm how many matches remain after dropping invalid rows
print("Training set (after drop) matches:", len(train_df))
print("Validation set (after drop) matches:", len(val_df))
print("Test set (after drop) matches:", len(test_df))


Training set (after drop) matches: 2905
Validation set (after drop) matches: 375
Test set (after drop) matches: 375


__Explanation__: We combined home and away data to compute each team's rolling 5-match average xG:

- We sorted combined_stats by team and date, then used groupby('Team') with a rolling window of 5. The shift(1) ensures that for a given match, we only average the previous 5 matches for that team. If a team has fewer than 5 prior matches in our dataset, the result remains NaN for those early games.

- We then filled the df DataFrame's new columns using the MatchID (which corresponds to the original match index) and whether that entry was a home or away record.

- After adding the new columns to df, we propagated them to our train_df, val_df, and test_df. We then dropped any rows where either home_team_avg_xG_last5 or away_team_avg_xG_last5 is NaN. This removes matches where one or both teams did not have 5 previous matches to compute an average (for example, matches in the first few weeks of a season or involving newly promoted teams without prior data).

- After dropping these, the training set size will shrink slightly (losing early-season games from 2015–2016 and 2016–2017 where xG history wasn't available for some teams), and the validation and test sets may lose a few matches (typically the first few matches of the season for teams with no prior Premier League data). We now have clean splits with the required features available for all remaining matches.

---
# 5. Select Features and Define the Target Variable
---

Now we prepare the feature matrix X and target vector y for modeling. Per the instructions, we will use the following features:

- ELO_Difference – the difference in Elo rating between the home team and away team (home minus away).

- home_team_avg_xG_last5 – the home team’s trailing 5-match average xG.

- away_team_avg_xG_last5 – the away team’s trailing 5-match average xG.

The target we predict will be whether the home team won the match. We'll create a binary target:

- y = 1 if the home team won (home win),

- y = 0 if the home team did not win (draw or away win).

Let's construct X and y for each of the train, validation, and test sets:


In [5]:
# Define the feature columns
features = ['ELO_Difference', 'home_team_avg_xG_last5', 'away_team_avg_xG_last5']

# Construct feature matrices for each subset
X_train = train_df[features].values
X_val   = val_df[features].values
X_test  = test_df[features].values

# Define the target: 1 if home win, 0 otherwise
y_train = (train_df['FTR'] == 'H').astype(int).values
y_val   = (val_df['FTR'] == 'H').astype(int).values
y_test  = (test_df['FTR'] == 'H').astype(int).values

# Quick sanity check on class balance in training data
print("Home wins in training set:", y_train.mean())


Home wins in training set: 0.4495697074010327


__Explanation__: We pull the three specified feature columns into NumPy arrays for modeling. The target FTR (Full Time Result) in the dataset is a categorical label 'H', 'D', or 'A' (home win, draw, away win). We convert this to a binary outcome where 1 indicates a home win (FTR = 'H') and 0 indicates otherwise. The quick sanity check prints the proportion of home wins in the training set (for example, ~0.45 would mean 45% of training games were home wins), giving us a sense of class distribution. 

---
## 6. Standardize Features (Using Only Training Data Stats)
---

- It is good practice to standardize features (mean-center and scale to unit variance) for logistic regression, especially when features are on different scales. We must be careful to fit the scaler only on the training data, and then apply that transformation to validation and test sets, to avoid any look-ahead bias.

We'll use StandardScaler from scikit-learn for this:

In [8]:
from sklearn.preprocessing import StandardScaler

# Fit a scaler on the training features
scaler = StandardScaler()
scaler.fit(X_train)

# Transform the features of train, val, and test sets
X_train_scaled = scaler.transform(X_train)
X_val_scaled   = scaler.transform(X_val)
X_test_scaled  = scaler.transform(X_test)

__Explanation__: We fit the StandardScaler on X_train only. This computes the mean and standard deviation of each feature using the training data. Then we transform all sets with the same scaler. This ensures that the validation and test data are scaled using parameters derived from training data only (simulating how a model in production would handle new data). Now all features are on a comparable scale, which helps the logistic regression model converge and also allows the regularization to treat features more equally.

---
## 7. Train Logistic Regression Models with Various Regularization Strengths
---

- We will train multiple logistic regression models with different regularization strengths (inverse regularization parameter C) to find the best one. The regularization used by default in scikit-learn's LogisticRegression is L2 (ridge), and C is the inverse of the regularization strength (so a smaller C means stronger regularization, and a larger C means weaker regularization).

- We will try a range of C values (e.g., 0.01, 0.1, 1, 10, 100) and evaluate each model on the validation set. We'll use the F1-score on the validation set as the metric to select the best model (per the instructions).

In [10]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

# Define a range of C values to try
C_values = [0.01, 0.1, 1, 10, 100]

best_model = None
best_C = None
best_val_f1 = -1.0

print("Validation F1-scores for different regularization strengths:")
for C in C_values:
    # Train logistic regression with given C (using lbfgs solver for stability)
    model = LogisticRegression(C=C, solver='lbfgs', max_iter=1000)
    model.fit(X_train_scaled, y_train)
    
    # Predict on validation set and compute F1-score
    y_val_pred = model.predict(X_val_scaled)
    val_f1 = f1_score(y_val, y_val_pred)
    print(f"  C = {C}: F1 = {val_f1:.3f}")
    
    # Track the best model based on validation F1
    if val_f1 > best_val_f1:
        best_val_f1 = val_f1
        best_C = C
        best_model = model

print(f"Best regularization C = {best_C}, which gave validation F1 = {best_val_f1:.6f}")


Validation F1-scores for different regularization strengths:
  C = 0.01: F1 = 0.602
  C = 0.1: F1 = 0.618
  C = 1: F1 = 0.621
  C = 10: F1 = 0.621
  C = 100: F1 = 0.621
Best regularization C = 1, which gave validation F1 = 0.620690


__Explanation__: We loop over the candidate C values:

- For each C, we train a logistic regression model on the scaled training data.

- We then predict the validation set outcomes and calculate the F1-score for the positive class (home win).

- We print each result for transparency. Typically, we expect some C in the middle of the range to perform best; too high C (very little regularization) or too low C (too much regularization) might hurt performance.

- We keep track of the model with the highest validation F1. After the loop, best_model holds the selected logistic regression model, and best_C is its regularization strength.

---
## 8. Select the Best Model Using Validation F1-Score
---

From the output above, we identify the C that yielded the highest F1-score on the validation set. We have already captured that in best_model.

Let's assume (for example) that the best C turned out to be 1.0 based on the printed results (the actual output above will show the F1 for each and the best selection). The model corresponding to C=1.0 is now considered our tuned model.

We will proceed using best_model as the final chosen model. (In a real scenario, one might consider re-training a fresh model on the combination of training+validation data with this hyperparameter, but here we'll continue with the model trained on the training set for evaluation.)

---
## 
---

Now we evaluate how well this chosen model performs on the test set (the 2024–2025 season, which the model has never seen). We'll compute:

- __Accuracy__ – the overall proportion of correct predictions.

- __F1-score__ – the harmonic mean of precision and recall for the home-win class (this focuses on how well we predicted "home wins").

- __Confusion Matrix__ – to see the breakdown of predictions vs actual outcomes.

In [16]:
from sklearn.metrics import accuracy_score, confusion_matrix

# Predict on the test set
y_test_pred = best_model.predict(X_test_scaled)

# Compute evaluation metrics
test_accuracy = accuracy_score(y_test, y_test_pred)
test_f1 = f1_score(y_test, y_test_pred)
cm = confusion_matrix(y_test, y_test_pred)

print(f"Test Accuracy: {test_accuracy:.6f}")
print(f"Test F1-score: {test_f1:.6f}")
print("Confusion Matrix (rows=true, cols=pred):")
print(cm)

Test Accuracy: 0.669333
Test F1-score: 0.583893
Confusion Matrix (rows=true, cols=pred):
[[164  57]
 [ 67  87]]


__Explanation__: We use the model to predict home win (1) or not (0) for each test match. The accuracy gives a general sense of performance (e.g., an accuracy of 0.67 would mean the model got 67% of matches correct overall). The F1-score specifically for home wins tells us how well the model balances precision and recall for predicting a home victory – this is useful if the classes are imbalanced (there are typically slightly more non-home-wins than home wins, since draws and away wins combined outnumber home wins slightly).

The confusion matrix output is a 2x2 array:

- [0,0]: True negatives (actual was 0 = not a home win, model predicted 0).

- [0,1]: False positives (actual was not a home win, model predicted home win).

- [1,0]: False negatives (actual was home win, model predicted not a home win).

- [1,1]: True positives (actual home win, model predicted home win).

For example, a confusion matrix of [[TN, FP], [FN, TP]] = [[164, 57], [67, 87]] would mean:

- 164 matches where it was not a home win and the model correctly predicted that.

- 57 matches where it was not a home win but the model incorrectly predicted a home win.

- 67 matches where the home team did win but the model missed it.

- 87 matches where the home team won and the model correctly predicted a win.

From these, we could derive that in this scenario the accuracy is $(164+87) / (164+57+67+87)$ and the F1-score corresponds to the precision and recall for the 87 true positives out of predicted and actual positives.

---
## 10. Interpret the Model Coefficients
---

Finally, let's interpret what the logistic regression has learned from the data. We will examine the coefficients for each feature in the model:

In [17]:
# Extract the coefficients and intercept from the best model
feature_names = features  # ['ELO_Difference', 'home_team_avg_xG_last5', 'away_team_avg_xG_last5']
coefs = best_model.coef_[0]    # coefficients for our three features
intercept = best_model.intercept_[0]

# Display the coefficients alongside feature names
for name, coef in zip(feature_names, coefs):
    print(f"{name}: {coef:.3f}")
print(f"Intercept: {intercept:.3f}")

ELO_Difference: 0.755
home_team_avg_xG_last5: 0.080
away_team_avg_xG_last5: -0.159
Intercept: -0.238


For correlation coefficients:
- __+1__ means perfect positive correlation between the predictor and the target variable
- __0__ means there is no correlation between the predictor and the target variable
- __-1__ means perfect negative correlation between the predictor and the target variable

__Explanation__: The logistic regression coefficients (learned on standardized features) tell us the direction and relative importance of each predictor:

- __ELO_Difference__: This coefficient is typically positive and by far the largest. A positive coefficient means that as the home team's Elo rating advantage increases (home Elo minus away Elo is larger), the log-odds of a home win increase. In other words, if the home team is stronger than the away team (higher Elo), the model strongly favors a home win. For example, if this coefficient is around +0.75 (after standardization), it has a substantial impact, indicating Elo difference is a very influential predictor.

- __home_team_avg_xG_last5__: This coefficient is usually positive (though likely smaller in magnitude than Elo). A positive coefficient here means if the home team has been performing well offensively in recent matches (higher average xG in last 5 games), it increases the chances of a home win. In our results, this coefficient might be relatively small (e.g., around +0.08 in standardized terms), suggesting a modest effect: recent home team form matters, but not as much as Elo.

- __away_team_avg_xG_last5__: This coefficient is typically negative. A negative value means that if the away team has a high recent average xG (i.e., the away team has been in good attacking form), it decreases the likelihood of a home win (which makes sense – a strong away team performance history implies the away team could do well, thus reducing the home team's win probability). For instance, a coefficient around -0.16 (standardized) would indicate that an increase of one standard deviation in the away team's trailing xG average lowers the log-odds of a home win by 0.16. This effect is present but again smaller than the Elo effect.

The __intercept__ term (e.g., around -0.24) represents the baseline log-odds of a home win when all features are at their mean values (since we standardized features, mean is 0). A slightly negative intercept in this case might correspond to the fact that, all else equal, the probability of home win is a bit less than 50% (which aligns with real-world data where home wins are somewhat less frequent than the combined probability of away win or draw).

In summary, __Elo rating difference__ is the strongest predictor in our model of home wins, which aligns with domain expectations (team strength is crucial). The trailing 5-match xG averages provide additional context: a home team in good attacking form and/or an away team in poor form can tilt the odds towards a home win, whereas a high-performing away team can tilt it away. All coefficients align with intuition: stronger home team and better recent home form increase win likelihood, while strong away team recent form decreases it. This interpretation gives us confidence that the model is capturing meaningful patterns in the data.